In [124]:
from pprint import pprint
from nltk.tag import hmm
from sklearn.externals import joblib
from nltk.tag.hmm  import HiddenMarkovModelTagger, HiddenMarkovModelTrainer
from nltk.probability import LidstoneProbDist
from nltk.tokenize import sent_tokenize

import nltk
import dill
import pickle
import os
import pandas as pd
import numpy as np
import sys

import re
import gensim
import nltk
import math
import pickle
from dataPreprocessing import getData

sys.path.append("lib/uuparser/barchybrid/src/")
import utils
from arc_hybrid import ArcHybridLSTM

In [125]:
source_list = ['dataset/training_text.csv', 'dataset/Indonesian_Tweets.tsv']

def dataGetter(source_list):
    corpus = []
    for source in source_list:
        corpus = corpus + getData(source)
    return corpus

In [126]:
class similarity:
    def __init__(self, source_list):
        corpus = dataGetter(source_list)
        self.model = self.getWord2Vec(corpus)
        self.aspects = ['makanan', 'pelayanan', 'harga', 'suasana']
        self.polarities = ['baik', 'buruk']
        
    def getWord2Vec(self, toFeed, dim=50):
        return gensim.models.Word2Vec(toFeed, min_count=1,  size=dim)
    
    def most_similar_aspect(self, word):
        most_similar = (0, "")
        for aspect in self.aspects:
            score = self.model.wv.similarity(word, aspect)
            if score > most_similar[0]:
                most_similar = (score, aspect)
        return most_similar
    
    def most_similar_polarities(self, word):
        most_similar = (0, "")
        for polarity in self.polarities:
            score = self.model.wv.similarity(word, polarity)
            if score > most_similar[0]:
                most_similar = (score, polarity)
        return most_similar
    
    def most_similar_word_with_aspect(self, sentence, aspect):
        most_similar = (0, "")
        for word in sentence.split():
            score = self.model.wv.similarity(word, aspect)
            if score > most_similar[0]:
                most_similar = (score, word)
        return most_similar
    
def save_model(filename, obj):
    with open(filename, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
def load_model(filename):
    with open(filename, 'rb') as handle:
        model = pickle.load(handle)
    return model
    

In [127]:
sim = None
try:
    sim = load_model('model.pickle')
except:
    sim = similarity(source_list)
    save_model('model.pickle', sim)

In [4]:
MODELS_DIR = 'models/id_gsd/'
TAGGER_FILE_NAME = 'tagger.dill'
PARAMS_FILE = MODELS_DIR + "params.pickle"
PARSER_FILE="barchybrid.model"

In [5]:
with open(MODELS_DIR + TAGGER_FILE_NAME, 'rb') as f:
    hmm_tagger = dill.load(f)

In [6]:
with open(PARAMS_FILE, 'r') as paramsfp:
    words, w2i, pos, rels, cpos, langs, stored_opt, ch = pickle.load(paramsfp)
    parser = ArcHybridLSTM(words, pos, rels, cpos, langs, w2i,
                           ch, stored_opt)
    model = os.path.join(MODELS_DIR, PARSER_FILE)
    parser.Load(model)

Loading model from models/id_gsd/barchybrid.model


# PROCESS INDIVIDUAL REVIEW

contoh review:

sapi bakarnya enak banget harganya juga lumayan murah

In [7]:
FOOD_POSITIVE_ADJ = ['enak', 'banyak']
FOOD_NEGATIVE_ADJ = ['mahal']

PRICE_POSITIVE_ADJ = ['murah', 'banyak']
PRICE_NEGATIVE_ADJ = ['mahal', 'sedikit']

SERVICE_POSITIVE_ADJ = ['cepat', 'baik', 'bagus', 'ramah']
SERVICE_NEGATIVE_ADJ = ['kasar', 'lambat']

AMBIENCE_POSITIVE_ADJ = ['murah']
AMBIENCE_NEGATIVE_ADJ = ['mahal']

In [180]:
review = "Tadi gue first time lewat Beji pas lewat dpn cafe ini, gue lgsng trtarik buat dtg Karna baru bgt buka. Gue mesen beberapa makanan yaitu roti choco crunch , sossis and potato pke sauce bbq apa tau namanya lupa, sm pizza mie pake topping smoked beef. Gilak gue rasa utk servis msh hrs byk belajar deh, gak kyk td pelayannya gak peka. Masih kaku gt. Terus masa gue dah hampir 30menit lama gak ada satupun mknan yg dtg. Trs cwok gue manggil waittersnya nanya mknannya dah jd apa belum, dan mas2 nya blg ""maaf ya mas td kita bingung cari yg mesen"" buset deh trnyata bener meja sblah gue kyknya dah mulai bosen. Pelayannya pun terlihat pd masih kebingungan. So klo mnrt gue dri segi makanan sih lumayan, service dan prosedur pelayanannya sih yg mesti jd concern, kliatan bgt blm ready. Pdhal tempatnya dah keren, luas bgt. Interiornya keren. Smga bisa terus maju."

In [177]:
point1 = ["VBD", "VB", "VBG", "VBN","VBP", "VBZ", "JJ", "JJR", "JJS", "RB", "RBR", "RBS"]
point2 = ["JJ", "JJR", "JJS", "RB", "RBR", "RBS"]
verb = ["VBD", "VB", "VBG", "VBN","VBP", "VBZ"]
noun = ["NN", "NNS", "NNP", "NNPS", "Z"]
adverb =["RB", "RBR", "RBS"]
adjective = ["JJ", "JJR", "JJS"]

In [181]:
def examine_polarity(sentence):
    """
        Check negative positive word regarding food, price, service, and ambience aspect in particular sentence
    """
    polarities = {
        'makanan': 0,
        'harga': 0,
        'pelayanan': 0,
        'suasana': 0,      
    }
    
    tagged_sentence = ' '.join(['{0}/{1}'.format(word, tag) for word, tag in hmm_tagger.tag(review.split())])
    data = utils.read_conll_text(tagged_sentence)
    pred = parser.Predict(data)
    depedency_tree = list(pred)[0]
    
    for p in depedency_tree:
        if p.pos == 'JJ':
            try:
                pol = sim.most_similar_polarities(p.form)
                weight = 1 if (pol[1] == 'baik' or pol[0] < 0.5) else -1
                parent_id = p.pred_parent_id
                offset = 5
                while(parent_id != None and parent_id > 0):
                    parent = depedency_tree[parent_id]
                    if (parent.pos in noun):
                        aspect = sim.most_similar_aspect(parent.form)
                        polarities[aspect[1]] += weight
                        break
                    parent_id = parent.pred_parent_id
                    offset -= 1
            except:
                print ""
    return polarities['makanan'], polarities['harga'], polarities['pelayanan'], polarities['suasana'] 

In [186]:
def extract_review_polarities(review):
    polarities = {
        'makanan': 0,
        'harga': 0,
        'pelayanan': 0,
        'suasana': 0,      
    }
    sentences = sent_tokenize(review)
    for sentence in sentences:
        pol_food, pol_price, pol_service, pol_ambience = examine_polarity(sentence)
        polarities['makanan'] += pol_food
        polarities['harga'] += pol_price
        polarities['pelayanan'] += pol_service
        polarities['suasana'] += pol_ambience
        
    for key, value in polarities.items():
        if value > 0:
            polarities[key] = 'POSITIVE'
        elif value < 0:
            polarities[key] = 'NEGATIVE'
        else:
            del polarities[key]
    return polarities


In [187]:
extract_review_polarities(review)

Reading Tadi/NN gue/FW first/FW time/FW lewat/FW Beji/FW pas/FW lewat/FW dpn/FW cafe/FW ini,/FW gue/FW lgsng/FW trtarik/FW buat/FW dtg/FW Karna/FW baru/JJ bgt/SYM buka./CD Gue/CD mesen/CD beberapa/CD makanan/NN yaitu/SC roti/FW choco/FW crunch/FW ,/Z sossis/FW and/FW potato/FW pke/FW sauce/FW bbq/FW apa/FW tau/FW namanya/FW lupa,/FW sm/FW pizza/FW mie/FW pake/FW topping/FW smoked/FW beef./FW Gilak/FW gue/FW rasa/FW utk/FW servis/FW msh/FW hrs/FW byk/FW belajar/FW deh,/FW gak/FW kyk/FW td/FW pelayannya/FW gak/FW peka./FW Masih/FW kaku/FW gt./FW Terus/FW masa/FW gue/FW dah/FW hampir/RB 30menit/RB lama/JJ gak/SC ada/VB satupun/FW mknan/FW yg/FW dtg./FW Trs/FW cwok/FW gue/FW manggil/FW waittersnya/FW nanya/FW mknannya/FW dah/FW jd/FW apa/FW belum,/FW dan/CC mas2/FW nya/FW blg/FW maaf/FW ya/FW mas/FW td/FW kita/FW bingung/FW cari/FW yg/FW mesen/FW buset/FW deh/FW trnyata/FW bener/FW meja/FW sblah/FW gue/FW kyknya/FW dah/FW mulai/VB bosen./PRP Pelayannya/NEG pun/RP terlihat/VB pd/PRP masih/M

Time: 0.36s


Reading Tadi/NN gue/FW first/FW time/FW lewat/FW Beji/FW pas/FW lewat/FW dpn/FW cafe/FW ini,/FW gue/FW lgsng/FW trtarik/FW buat/FW dtg/FW Karna/FW baru/JJ bgt/SYM buka./CD Gue/CD mesen/CD beberapa/CD makanan/NN yaitu/SC roti/FW choco/FW crunch/FW ,/Z sossis/FW and/FW potato/FW pke/FW sauce/FW bbq/FW apa/FW tau/FW namanya/FW lupa,/FW sm/FW pizza/FW mie/FW pake/FW topping/FW smoked/FW beef./FW Gilak/FW gue/FW rasa/FW utk/FW servis/FW msh/FW hrs/FW byk/FW belajar/FW deh,/FW gak/FW kyk/FW td/FW pelayannya/FW gak/FW peka./FW Masih/FW kaku/FW gt./FW Terus/FW masa/FW gue/FW dah/FW hampir/RB 30menit/RB lama/JJ gak/SC ada/VB satupun/FW mknan/FW yg/FW dtg./FW Trs/FW cwok/FW gue/FW manggil/FW waittersnya/FW nanya/FW mknannya/FW dah/FW jd/FW apa/FW belum,/FW dan/CC mas2/FW nya/FW blg/FW maaf/FW ya/FW mas/FW td/FW kita/FW bingung/FW cari/FW yg/FW mesen/FW buset/FW deh/FW trnyata/FW bener/FW meja/FW sblah/FW gue/FW kyknya/FW dah/FW mulai/VB bosen./PRP Pelayannya/NEG pun/RP terlihat/VB 

{'makanan': 'POSITIVE'}

In [184]:
sim = load_model('model.pickle')